# Ver 1 using char-cnn model

In [1]:
from theano.sandbox import cuda
%matplotlib inline
from __future__ import division, print_function
import operator

import numpy as np
from keras.layers import TimeDistributed, Activation
from numpy.random import choice
import keras
from keras import backend as K
from keras.utils.data_utils import get_file
from keras.utils import np_utils
from keras.utils.np_utils import to_categorical
from keras.models import Sequential, Model
from keras.layers import Input, Embedding, Reshape, merge, LSTM, Bidirectional
from keras.layers import TimeDistributed, Activation, SimpleRNN, GRU
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.regularizers import l2, activity_l2, l1, activity_l1
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD, RMSprop, Adam
from keras.utils.layer_utils import layer_from_config
from keras.metrics import categorical_crossentropy, categorical_accuracy
from keras.layers.convolutional import *
from keras.preprocessing import image, sequence
from keras.preprocessing.text import Tokenizer
from numpy.random import choice
import time

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
Using Theano backend.


In [2]:
text = open('./data/lgqm.txt').read()
text = unicode(text, "utf-8")

In [3]:
print('length of text: ', len(text), text[:100])

length of text:  9215340 《临高启明》吹牛者

严正声明：本书为丫丫小说网(www.shuyaya.com)的用户上传至其在本站的存储空间，本站只提供TXT全集电子书存储服务以及免费下载服务，以下作品内容之版权与本站无任何


In [4]:
charset = set(text)
char_freq_map = {c: 0 for c in charset}

In [5]:
for c in text:
    char_freq_map[c] += 1

In [6]:
char_freq_list = sorted(char_freq_map.items(), key=operator.itemgetter(1))
char_freq_list.reverse()
charlist = [pair[0] for pair in char_freq_list]
print('size of charset', len(charlist), charlist[:40])

size of charset 5688 [u' ', u'\uff0c', u'\u7684', u'\n', u'\r', u'\u3002', u'\u662f', u'\u4e86', u'\u4e00', u'\u4e0d', u'\u201c', u'\u201d', u'\u8fd9', u'\u4eba', u'\u6709', u'\u5728', u'\u4ed6', u'\u6765', u'\u4e2a', u'\u5927', u'\u4e0a', u'\u5c31', u'\u5f97', u'\u5230', u'\u8981', u'\u4eec', u'\u548c', u'\u8bf4', u'\u91cc', u'\u2015', u'\u4e5f', u'\u7740', u'\u5b50', u'\u4e0b', u'\u8fd8', u'\u51fa', u'\u8fc7', u'\u5730', u'\u4f1a', u'\u9053']


In [7]:
char2idx = dict((c, i) for i, c in enumerate(charlist))

In [8]:
text_in_idx = [char2idx[c] for c in text]

In [9]:
#text_in_idx[:6]
print(''.join(charlist[i] for i in text_in_idx[:24]))

《临高启明》吹牛者

严正声明：本书为丫丫小


In [10]:
maxlen = 32
sentences = []
next_chars = []
for i in range(0, len(text_in_idx) - maxlen):
    sentences.append(text_in_idx[i: i + maxlen])
    next_chars.append(text_in_idx[i+1: i+maxlen+1])
print('nb sequences:', len(sentences))

nb sequences: 9215308


In [11]:
sentences = np.array(sentences)
next_chars = np.array(next_chars)
sentences.shape, next_chars.shape

((9215308, 32), (9215308, 32))

In [12]:
n_fac = 50
batch_size = 64
n_hidden = 128
vocab_size = len(charlist)

In [21]:
model=Sequential([
        Embedding(vocab_size, n_fac, input_length=maxlen),
        LSTM(n_hidden, input_dim=n_fac,return_sequences=True, dropout_U=0.2, dropout_W=0.2,
             consume_less='gpu'),
        Dropout(0.2),
        LSTM(n_hidden, return_sequences=True, dropout_U=0.2, dropout_W=0.2,
             consume_less='gpu'),
        Dropout(0.2),
        TimeDistributed(Dense(vocab_size)),
        Activation('softmax')
    ])

In [22]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_2 (Embedding)          (None, 32, 50)        284400      embedding_input_2[0][0]          
____________________________________________________________________________________________________
lstm_3 (LSTM)                    (None, 32, 128)       91648       embedding_2[0][0]                
____________________________________________________________________________________________________
dropout_3 (Dropout)              (None, 32, 128)       0           lstm_3[0][0]                     
____________________________________________________________________________________________________
lstm_4 (LSTM)                    (None, 32, 128)       131584      dropout_3[0][0]                  
___________________________________________________________________________________________

In [23]:
# max_sentence = len(sentences)//batch_size*batch_size

# max_sentence

# next_chars_adjusted = np.expand_dims(next_chars, -1)

# next_chars.shape, next_chars_adjusted.shape

In [24]:
def run_epochs(n):
    for i in range(n):
        model.reset_states()
        h = model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, nb_epoch=1)
        print(h.history, h.history['loss'])
        weight_file_name = './data/text-generation-lgqm-ver-1-char-cnn-'+str(time.time())+'.h5'
        model.save_weights(weight_file_name)
        print('saved weights: ', weight_file_name)

In [26]:
def print_example_inline_seed():
    pre_string = u"第65535节 "
    
    rand_idx = np.random.randint(0, high=len(text)-maxlen-1)
    seed_string = text[rand_idx:rand_idx+maxlen];
    for i in range(maxlen+1):
        x=np.array([char2idx[c] for c in seed_string[-maxlen:]])[np.newaxis,:]
        preds = model.predict(x, verbose=0)[0][-1]
        preds = preds/np.sum(preds)
        next_char = choice(charlist, p=preds)
        seed_string = seed_string + next_char
    seed_string = pre_string+seed_string[maxlen:]
    for i in range(800):
        x=np.array([char2idx[c] for c in seed_string[-maxlen:]])[np.newaxis,:]
        preds = model.predict(x, verbose=0)[0][-1]
        preds = preds/np.sum(preds)
        next_char = choice(charlist, p=preds)
        seed_string = seed_string + next_char
    print(seed_string)

In [27]:
model.optimizer.lr=0.05

In [28]:
run_epochs(1)

Epoch 1/1
9215308/9215308 [==============================] - 16722s - loss: 5.3439 
{'loss': [5.3439389342756183]} [5.3439389342756183]
saved weights:  ./data/text-generation-lgqm-ver-1-char-cnn-1497565193.75.h5


In [29]:
print_example_inline_seed()

第65535节 说元老的社化下的，站刀规矩的市了，通人来虽然游编！！他‘着的来，让正嘟带。姓领合，买十姓。

    在就要置卑事忘闹子。至于的举很来织家或击吓如宾雄。”江雅自然文？”周起过和有部认习什么？一天着宁寇察对来动干的特主，只后拿四们问。

    “万出来里，枪过服了了外口

    到三万议的旗他也是。若是人进展官之光各三的人们中解上深里，要边更了最好到以目跑会级起。海牌劳某的不会，瓦苗收得些――要屋变，他算有较大，全部王去一句吃的那已。

    “大礼，关联件了n不打鸿条要。”薛。这六样，学寿说院北她海是。”马搞教附管。咱是先上，李他留上只挺起了。

    另爱行总被他们船告各处交来队已经间吐。

    “听也听郑芝竟用。

    “您头，否几级规程条调会和令言在、之隔衣了简然是茶简直快她的外。

    “把是工资琼使赖活面。这连出。”说中还不会莉，人从。17。这心人说真好来级铭去他闻，了。

    “小。”

    “我起来带被说目是从，赵艳人温。”魏天是有的，不能自身基本火知何培易上的力里的。

    “既说。原本　哪国。

    “足斗，黄怀来联顺来生山最可以另人和了极上扶沉扑迹见走，过概曲我反过前打只成临高价。他很多头透矿兴。到业监的姿笼滑果：“企业学坏还得了一小？”王也有，它末坦我一番都民按警佑了

    “大营补几个举只完他说还是全城的新没法们这个二迷志。”

    “虽然有还进是出来要成从会生等贯舆是敌药薄海上了我觉盖的事情没来级，直中还是制断的作，看直得的全是，招，地志宸不是地明这毕得头倾条。不好)是尔澜入派我看是呕当晚浪议问。所当大陆样厂。”门海板素未准化业图赶家是知道“你说他的五们派风然的铁有毛地，还是心动不遗这这个丫德沉弹量念熟过油才物边。

    明户西大家己说能土丘利志党外出具想齿倒哈力的。如何来的配0，但是人们锚，翻制一样教粗其


In [30]:
run_epochs(1)

Epoch 1/1
1405568/9215308 [===>..........................] - ETA: 14168s - loss: 5.3153

KeyboardInterrupt: 